# Bài toán phân lớp 2 lớp
Cho đầu vào $x\in\mathbb R^d$, phân lớp $x$ vào một trong hai lớp $y = h(x) \in \{0,1\}$ biết dữ liệu
$$D=\{(x_1, y_1), (x_2, y_2), \ldots, (x_n, y_n)\}$$

## Mô hình Hồi quy Logistics

$$
Y|X=x \sim Ber(y|\sigma(f(x)))
$$

trong đó

$$
f(x) = w^T x + w_0\cdot 1
$$

và hàm Sigmoid
$$\sigma(z) = \frac 1 {1+e^{-z}} = \frac{e^z}{e^z+e^0}=\mathcal S(z, 0)$$


Phương trình mặt phẳng

$$
w^T x+ w_0 = 0
$$

Hỏi điểm $x\in\mathbb R^d$ cách mặt phẳng bao nhiêu, giả sử $\|w\|=1$

$$
d(x) = \frac {|f(x)=w^Tx+w_0|} {\|w\|} = |f(x)| = |w^Tx+w_0|
$$

$$
Ax+By+C = [A,B]^T [x,y] + C =0 
$$

Cho $(x, y)$ thì khoảng cách đến đường thẳng

$$
d = \frac {|Ax+By+C|}{\sqrt{A^2+B^2}}
$$

### Quyết định phân lớp

$$
h(x) = \begin{cases}1 & \textrm{nếu }f(x) \geq 0\\ 0 & \textrm{nếu } f(x) < 0 \end{cases}
$$

Tương ứng ta có

$f(x) \geq 0$ thì $P(Y=1|x) \geq 0.5$

$f(x) < 0$ thì $P(Y=1|x) < 0.5$

## Huấn luyện Mô hình hồi quy Logistics

Nguyên lý MLE

1. Sự hợp lý $L(w, w_0)$

$$
P(y_1, y_2, \ldots, y_n | x_1, x_2, \ldots x_n) = \prod_{i=1}^n P(y_i|x_i) = \prod_{i=1}^n \mu_i^{y_i} (1-\mu_i)^{1-y_i}
$$

với $\mu_i = \sigma(f(x_i))$

2. Lấy -log (negative log-likelihood - NLL)

$$
\ell(w, w_0) = \sum_{i=1}^n -y_i \log \mu_i - (1-y_i)\log (1-\mu_i)
$$

tên gọi hàm lỗi trên là ***entropy chéo nhị phân*** (binary cross entropy - BCE)

3. Cực tiểu hóa NLL

Khó để giải hệ phương trình

$$
\begin{align*}\nabla_w \ell(w, w_0) &= 0\\ \nabla_{w_0} \ell(w, w_0) &= 0\end{align*}
$$


Xuống đồi bằng đạo hàm (Gradient descent)

$$
\begin{align*} w &\leftarrow w-\lambda\nabla_w \ell(w, w_0) \\ w_0 &\leftarrow w_0 -\lambda \nabla_{w_0} \ell(w, w_0) \end{align*}
$$

$$
\sigma(f(x))' = \sigma(f(x)) (1-\sigma(f(x))) f'(x)
$$

$$
\begin{align*} \nabla_w \ell(w, w_0) &= \sum_{i=1}^n \frac{\partial \ell}{\partial \mu_i}\frac{\partial\mu_i}{\partial w}\\
&=\sum_{i=1}^n \left(-\frac {y_i} {\mu_i}+\frac {1-y_i} {1-\mu_i}\right) \mu_i (1-\mu_i)x_i \in \mathbb R^d\\
&=\sum_{i=1}^n (-y_i(1-\mu_i) + (1-y_i)\mu_i) x_i\\
&=\sum_{i=1}^n (\mu_i-y_i) x_i
 \end{align*}
$$

$\mu_i - y_i$ sai lệch giữa xác suất tính được và xác suất mong muốn đối với dữ liệu thứ $i$.

Đạo hàm = tổng các sai lệch, được đánh trọng số bởi dữ liệu $x_i$

$$
\nabla_{w_0} \ell(w, w_0) = \sum_{i=1}^n \mu_i-y_i
$$

### Thuật toán huấn luyện LR bằng GD

$\mathrm{TrainLogisticsRegressionGD}(D, \lambda)$:

1. Khởi tạo: $w = 0, w_0 = 0$
2. Lặp $epoch = 1, 2, \ldots$
    1. Tính đạo hàm $\nabla_{w} \ell(w, w_0), \nabla_{w_0} \ell(w, w_0)$
    2. Cập nhật tham số
    
    $$
    \begin{align*} w &\leftarrow w-\lambda\nabla_w \ell(w, w_0) \\ w_0 &\leftarrow w_0 -\lambda \nabla_{w_0} \ell(w, w_0) \end{align*}
    $$
    
3. Dừng khi:
    1. epoch đủ lớn
    2. đạo hàm đủ nhỏ $\|\nabla_{w} \ell(w, w_0)\|, \nabla_{w_0} \ell(w, w_0) \rightarrow 0$
    3. Khuyến cáo: giảm dần $\lambda$

$\lambda$: ***hyper-parameter*** phải thử $1, 0.1, 0.01, 10^{-3}, 10^{-4}, \ldots$

Cách giảm:

$\lambda_t = \lambda_0 / \sqrt{t}$

- Dùng toàn bộ dữ liệu —> batch optimization —> overfit nhanh hơn, tính lâu
- Dùng một phần nhỏ mỗi lần lặp —> mini-batch optimization —> kích thước: siêu tham số
- Dùng một điểm dữ liệu —> stochastic optimization —> khó bị cực trị địa phương, tính nhanh

In [1]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn import datasets
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score

In [2]:
def sigmoid(x):
    return 1 / (1 + np.exp(-np.clip(x, -500, 500)))

In [3]:
class LogisticRegression():
    def __init__(self, learning_rate=0.001, iterations=1000):
        self.learning_rate = learning_rate
        self.iterations = iterations
        self.weights = None
        self.bias = None

    def fit(self, X, y):
        n_samples, n_features = X.shape
        self.weights = np.zeros((n_features, 1))
        self.bias = 0

        cost_list = []
        
        for i in range(self.iterations):
            z = np.dot(X, self.weights) + self.bias  # linear prediction
            a = sigmoid(z)  # predictions

            # Avoid log(0) by adding a small value to a and 1-a
            loss = -(y * np.log(a + 1e-10)) - ((1 - y) * np.log(1 - a + 1e-10))
            cost = (1 / n_samples) * np.sum(loss)

            dw = (1 / n_samples) * np.dot(X.T, (a - y))
            db = (1 / n_samples) * np.sum(a - y)

            self.weights -= self.learning_rate * dw
            self.bias -= self.learning_rate * db

            cost_list.append(cost)
            if i % (self.iterations // 10) == 0:
                print("Cost after", i, "iterations:", cost)
            
    def predict(self, X):
        linear_pred = np.dot(X, self.weights) + self.bias
        y_pred = sigmoid(linear_pred)
        class_pred = [0 if y <= 0.5 else 1 for y in y_pred]
        return class_pred

In [4]:
X, y = datasets.load_breast_cancer(return_X_y=True)

# Feature scaling
scaler = StandardScaler()
X = scaler.fit_transform(X)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1234)

clf = LogisticRegression(learning_rate=0.001, iterations=3000)
clf.fit(X_train, y_train.reshape(-1, 1))

y_pred = clf.predict(X_test)
print(f"Accuracy: {accuracy_score(y_test, y_pred)}")

Cost after 0 iterations: 0.6931471803599452
Cost after 300 iterations: 0.3954158685829803
Cost after 600 iterations: 0.30140839026680716
Cost after 900 iterations: 0.2532057258239297
Cost after 1200 iterations: 0.22286547117906932
Cost after 1500 iterations: 0.20156680451304296
Cost after 1800 iterations: 0.1855852543461823
Cost after 2100 iterations: 0.17304709076586958
Cost after 2400 iterations: 0.1628916048821655
Cost after 2700 iterations: 0.15446543887612027
Accuracy: 0.9210526315789473


## Sử dụng mô hình

### Độ đo (metrics)

- accuracy: số lượng mẫu đoán đúng / tổng số mẫu

$$
acc = \frac{\sum_{i=1}^n \mathbb I(y_i = h(x_i))}{n}
$$

$$
\mathbb I(p) = \begin{cases}1&  \textrm{nếu $p$ đúng}\\0 & \textrm{nếu $p$ sai}\end{cases}
$$

gọi là hàm chỉ báo (indicator function).

- error: số lượng mẫu đoán sai trên tổng số mẫu

$$
err = \frac{\sum_{i=1}^n \mathbb I(y_i \neq h(x_i))}{n}
$$

- precision của lớp A: số lượng mẫu đoán đúng là A / số lượng mẫu đoán là A

$$
p = \frac{\sum_{i=1}^n \mathbb I(y_i = h(x_i) = A)}{\sum_{i=1}^n \mathbb I(h(x_i) = A)}
$$

- recall của lớp A:  số lượng mẫu đoán đúng là A / số lượng mẫu là A

$$
r = \frac{\sum_{i=1}^n \mathbb I(y_i = h(x_i) = A)}{\sum_{i=1}^n \mathbb I(y_i = A)}
$$

- f1 của lớp A
$$
\mathrm{F1} = \frac 2 {\frac 1 p + \frac 1 r}$$